In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
class lstm_cell:

    def __init__(self, sess, seq_size, learn_rate, optimizer='adagrad'):
        self.sess = sess
        
        #self.ct_state = np.random.normal(size=[1, seq_size])
        #self.ht_state = np.random.normal(size=[1, seq_size])
        
        self.ct_state = np.zeros([1, seq_size])
        self.ht_state = np.zeros([1, seq_size])
        
        # Пара значений вход x и желаемый выход y.
        self.x = tf.placeholder(tf.float32, [1, seq_size])
        self.y = tf.placeholder(tf.float32, [1, seq_size])
        
        # Состояние сети. ht выходное значение, ct ячейка памяти.
        self.ct = tf.placeholder(tf.float32, [1, seq_size])
        self.ht = tf.placeholder(tf.float32, [1, seq_size])
        
        self.ft_wx,  self.ft_wh,  self.ft_b  = self.three(seq_size) # Фильтр забывания (forget gate).
        self.it_wx,  self.it_wh,  self.it_b  = self.three(seq_size) # Фильтр входа (input gate).
        self.ctt_wx, self.ctt_wh, self.ctt_b = self.three(seq_size) # Фильтр кандидатов (candidate gate).
        self.ot_wx,  self.ot_wh,  self.ot_b  = self.three(seq_size) # Фильтр выхода (output gate).
        
        self.ft = tf.math.sigmoid(tf.matmul(self.x, self.ft_wx) + tf.matmul(self.ht, self.ft_wh) + self.ft_b)
        self.it = tf.math.sigmoid(tf.matmul(self.x, self.it_wx) + tf.matmul(self.ht, self.it_wh) + self.it_b)
        self.ctt = tf.math.tanh(tf.matmul(self.x, self.ctt_wx) + tf.matmul(self.ht, self.ctt_wh) + self.ctt_b)
        self.ot = tf.math.sigmoid(tf.matmul(self.x, self.ot_wx) + tf.matmul(self.ht, self.ot_wh) + self.ot_b)
        
        self.ct_out = tf.math.multiply(self.ft, self.ct) + tf.math.multiply(self.it, self.ctt)
        self.ht_out = tf.math.multiply(self.ot, tf.math.tanh(self.ct_out))
        
        self.out_w = tf.Variable(tf.random.normal([seq_size, seq_size]), dtype=tf.float32)
        self.out_b = tf.Variable(tf.random.normal([1, seq_size]), dtype=tf.float32)
        self.out = tf.matmul(self.ht_out, self.out_w) + self.out_b
        
        self.cost = tf.reduce_mean(tf.square(self.out - self.y))
        
        if optimizer == 'adagrad':
            self.train_op = tf.train.AdagradOptimizer(learn_rate).minimize(self.cost)
        elif optimizer == 'adam':
            self.train_op = tf.train.AdamOptimizer().minimize(self.cost)
        elif optmizer == 'gradientdescent':
            self.train_op = tf.train.GradientDescentOptimizer(learn_rate).minimize(self.cost)
        
        sess.run(tf.global_variables_initializer())
        pass
    
    def clear_state(self):
        self.ct_state = np.zeros([1, seq_size])
        self.ht_state = np.zeros([1, seq_size])
    
    def three(self, seq_size):
        # Для LSTM распространённая практика bias задавать как 1.
        # https://datascience.stackexchange.com/questions/17987/how-should-the-bias-be-initialized-and-regularized
        return (tf.Variable(tf.random.normal([seq_size, seq_size]), dtype=tf.float32),
            tf.Variable(tf.random.normal([seq_size, seq_size]), dtype=tf.float32),
            tf.Variable(tf.zeros([1, seq_size]), dtype=tf.float32)) # <- попробуем на нулевом смещении
            #tf.Variable(tf.random.normal([1, seq_size]), dtype=tf.float32)) # <- пробуем на случайном смещении
            #tf.Variable(tf.ones([1, seq_size]), dtype=tf.float32)) # <- пробуем все 1
    
    def train(self, train_x, train_y, repeat):
        
        #for i in range(50):
            #self.ct_state, self.ht_state, _ = self.sess.run([self.ct_out, self.ht_out, self.train_op],
                #feed_dict={self.x:train_x, self.y:train_y, self.ct:self.ct_state, self.ht:self.ht_state})
                
        #self.ct_state, self.ht_state = self.sess.run([self.ct_out, self.ht_out],
            #feed_dict={self.x:train_x, self.ct:self.ct_state, self.ht:self.ht_state})
                    
        for i in range(repeat):
            
            #self.ct_state, self.ht_state, cost, _ = self.sess.run([self.ct_out, self.ht_out, self.cost, self.train_op],
                #feed_dict={self.x:train_x, self.y:train_y, self.ct:self.ct_state, self.ht:self.ht_state})
            
            cost, _ = self.sess.run([self.cost, self.train_op],
                feed_dict={self.x:train_x, self.y:train_y, self.ct:self.ct_state, self.ht:self.ht_state})
            
        self.ct_state, self.ht_state = self.sess.run([self.ct_out, self.ht_out],
            feed_dict={self.x:train_x, self.ct:self.ct_state, self.ht:self.ht_state})
            
        return cost
    
    def test(self, test_x):
        
        #self.ct_state, self.ht_state, out = self.sess.run([self.ct_out, self.ht_out, self.out],
            #feed_dict={self.x:test_x, self.ct:self.ct_state, self.ht:self.ht_state})

        #self.ct_state, self.ht_state = self.sess.run([self.ct_out, self.ht_out],
            #feed_dict={self.x:test_x, self.ct:self.ct_state, self.ht:self.ht_state})
                
        self.ct_state, self.ht_state, out = self.sess.run([self.ct_out, self.ht_out, self.out],
            feed_dict={self.x:test_x, self.ct:self.ct_state, self.ht:self.ht_state})
        
        #self.ct_state, self.ht_state = self.sess.run([self.ct_out, self.ht_out],
            #feed_dict={self.x:test_x, self.ct:self.ct_state, self.ht:self.ht_state})
        
        return out